In [1]:
import sqlite3
import pandas as pd
import os
import numpy as np

pd.set_option('display.max_column', None)

[Dataset for the task](https://www.kaggle.com/datasets/hugomathien/soccer)

In [2]:
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [3]:
players = pd.read_sql("SELECT * FROM Player;", db)

In [4]:
players.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


## Task 1

**Calculate the number of players with a height between 180 and 190 inclusive**

In [5]:
players_180_190 = players[(players.height >= 180) & (players.height <= 190)].player_api_id.nunique()

In [6]:
print('Number of players with a height between 180 and 190: {players_180_190} '
      .format(players_180_190=players_180_190))

Number of players with a height between 180 and 190: 5925 


## Task 2

**Calculate the number of players born in 1980**

In [7]:
# check column types
players.dtypes

id                      int64
player_api_id           int64
player_name            object
player_fifa_api_id      int64
birthday               object
height                float64
weight                  int64
dtype: object

In [8]:
# change type of birthday to DateTime
players.birthday = pd.to_datetime(players.birthday)

In [9]:
players_1980 = players[players.birthday.dt.year == 1980].player_api_id.nunique()

In [10]:
print('Number of players born in 1980: {players_1980}'
      .format(players_1980=players_1980))

Number of players born in 1980: 434


## Task 3

**Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.**

In [11]:
# find TOP-10 players with highest weight
players.sort_values(['weight', 'player_name'], ascending=[False, True]).head(10)

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
5901,5908,148325,Kristof van Hout,185306,1987-02-09,208.28,243
10297,10313,27313,Tim Wiese,53012,1981-12-17,193.04,243
4879,4884,5044,Jeroen Verhoeven,137692,1980-04-30,198.12,227
4408,4412,27267,Ishmael Miller,169725,1987-03-05,193.04,225
1595,1598,101584,Cassio,167958,1987-06-06,195.58,220
1819,1822,19020,Christopher Samba,46590,1984-03-28,193.04,220
5969,5976,210822,Lars Unnerstall,199833,1990-07-20,198.12,220
6762,6772,30669,Marcus Hahnemann,53785,1972-06-15,190.50,220
44,45,40005,Abdoulaye Faye,100329,1978-02-26,187.96,218
3026,3030,33060,Enoch Showunmi,157589,1982-04-21,195.58,218


In [12]:
highest_players = (players.sort_values(['weight', 'player_name'], ascending=[False, True]).head(10)
                   .player_name.tolist())

In [13]:
print('TOP-10 players with highest weight: {highest_players}'
     .format(highest_players=highest_players))

TOP-10 players with highest weight: ['Kristof van Hout', 'Tim Wiese', 'Jeroen Verhoeven', 'Ishmael Miller', 'Cassio', 'Christopher Samba', 'Lars Unnerstall', 'Marcus Hahnemann', 'Abdoulaye Faye', 'Enoch Showunmi']


## Task 4

**Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990.** <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [14]:
# find how many players was born each year
years = players.groupby(players.birthday.dt.year).player_api_id.nunique().sort_index(axis=0)

In [15]:
years_born_players = list(years.items())
print('Number of players born each year: {years}'
     .format(years=years_born_players))

Number of players born each year: [(1967, 1), (1968, 1), (1969, 6), (1970, 12), (1971, 12), (1972, 22), (1973, 43), (1974, 77), (1975, 113), (1976, 156), (1977, 191), (1978, 263), (1979, 345), (1980, 434), (1981, 475), (1982, 487), (1983, 541), (1984, 607), (1985, 593), (1986, 710), (1987, 722), (1988, 802), (1989, 723), (1990, 696), (1991, 699), (1992, 643), (1993, 529), (1994, 462), (1995, 331), (1996, 234), (1997, 103), (1998, 24), (1999, 3)]


## Task 5

**Calculate the mean and the standard deviation of the players' *height* with the name *Adriano*.** <br>
**Note:** Name is represented by the first part of `player_name`.

In [16]:
# find players with name Adriano
adriano = players[players.player_name.str.startswith('Adriano ')]
adriano.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
184,185,167648,Adriano De Pierro,193351,1991-01-11,185.42,170
185,186,280969,Adriano Facchini,205122,1983-03-12,187.96,172
186,187,39190,Adriano Ferreira Pinto,163653,1979-12-10,177.80,179
187,188,172172,Adriano Jose de Lara,193874,1987-12-09,185.42,190
188,189,39199,Adriano Pereira da Silva,135530,1982-04-03,180.34,165


In [17]:
adriano_mean, adriano_std = adriano.height.mean(), adriano.height.std()

In [18]:
print('''The mean height of players with name Adriano: {mean}.
The std of height of players with name Adriano: {std}.'''
     .format(mean=adriano_mean, std=adriano_std))

The mean height of players with name Adriano: 183.7266666666667.
The std of height of players with name Adriano: 3.824085075762125.


## Task 6

**How many players were born on each day of the week? Find the day of the week with the minimum number of players born.**

In [19]:
# find how many players was born on each  day
weekdays = players.groupby(players.birthday.dt.weekday).player_api_id.nunique()
weekdays

birthday
0    1643
1    1652
2    1607
3    1648
4    1685
5    1463
6    1362
Name: player_api_id, dtype: int64

In [20]:
dow_with_min_players_born = weekdays.idxmin()
dow_with_min_players_born

6

In [21]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

In [22]:
print('The minimum number of players was born on {day}'.format(day=days[dow_with_min_players_born]))

The minimum number of players was born on Sunday


## Task 7

**Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.**

In [23]:
# download League dataset
leagues = pd.read_sql("SELECT * FROM League;", db)
leagues.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


In [24]:
leagues.shape

(11, 3)

In [25]:
# download Match dataset
matches = pd.read_sql('SELECT * FROM Match', db)
matches.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.4,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [26]:
# calculate how many matches was played in each league
matches.groupby('league_id').match_api_id.nunique().sort_values(ascending=False)

league_id
21518    3040
4769     3040
1729     3040
10257    3017
13274    2448
7809     2448
17642    2052
15722    1920
19694    1824
1        1728
24558    1422
Name: match_api_id, dtype: int64

In [27]:
league_most_matches = matches.groupby('league_id').match_api_id.nunique().idxmax()
league_most_matches

1729

In [28]:
print('The league with the most matches in total is {league}'
     .format(league = leagues.loc[leagues.id == league_most_matches, 'name'].item()))

The league with the most matches in total is England Premier League


## Task 8

**Find a player who participated in the largest number of matches during the whole match history**

In [29]:
matches.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.4,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [30]:
# leave only colummns with player_api_id
player_match = matches.loc[:, 'home_player_1' : 'away_player_11']
player_match.head()

,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# create Series where will write all mentions of palyers' api id
one_column = pd.Series()

<ipython-input-31-e8021ecebb07>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  one_column = pd.Series()


In [32]:
list_of_columns = player_match.columns.tolist()
len(list_of_columns)

22

In [33]:
# write all mentions of palyers' api id
for column in list_of_columns:
    one_column = one_column.append(player_match[column])

In [34]:
one_column.shape

(571538,)

In [35]:
# drop all Nan values
one_column = one_column.dropna()
one_column.head()

144    39890.0
145    38327.0
146    95597.0
148    30934.0
149    37990.0
dtype: float64

In [36]:
int(one_column.value_counts().idxmax())

31293

In [37]:
players[players.player_api_id == int(one_column.value_counts().idxmax())]

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
9993,10009,31293,Steve Mandanda,163705,1985-03-28,185.42,181


In [38]:
max_matches_player = players.loc[players.player_api_id == int(one_column.value_counts().idxmax()), 'player_name'].item()

In [39]:
print('A player who participated in the largest number of matches is {name}.'
     .format(name=max_matches_player))

A player who participated in the largest number of matches is Steve Mandanda.


## Task 9

**List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [40]:
attributes = pd.read_sql("SELECT * FROM Player_Attributes;", db)
attributes.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [41]:
#leave only necessary columns
attributes_corr = attributes.iloc[:, 4:].drop(['preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], axis=1)
attributes_corr.head()

,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,62.0,66.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,61.0,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,61.0,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [42]:
# calculate correlation between columns
attributes_corr = attributes_corr.corr()
attributes_corr

,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
overall_rating,1.000000,0.766757,0.357699,0.329298,0.314099,0.458361,0.361470,0.354324,0.357772,0.349592,0.435018,0.444257,0.245655,0.254841,0.239896,0.769246,0.160364,0.427996,0.259023,0.327456,0.318661,0.392382,0.323934,0.250370,0.370019,0.431137,0.393189,0.133377,0.165349,0.128889,0.027976,0.004410,0.025682,0.005709,0.005687
potential,0.766757,1.000000,0.277755,0.286684,0.207706,0.382605,0.301302,0.338675,0.296073,0.263358,0.343716,0.401368,0.338044,0.340054,0.292950,0.580235,0.201974,0.325697,0.174819,0.260509,0.126931,0.312808,0.164815,0.165730,0.327829,0.378901,0.316414,0.055758,0.083822,0.063976,-0.011250,0.004742,0.089205,0.003406,0.003962
crossing,0.357699,0.277755,1.000000,0.576860,0.369747,0.790150,0.637891,0.809061,0.789132,0.707576,0.681959,0.807302,0.599012,0.579409,0.599248,0.384514,0.518518,0.656280,0.021320,0.564360,-0.070291,0.715925,0.323599,0.305952,0.682113,0.693227,0.571473,0.235159,0.285265,0.275467,-0.603898,-0.595610,-0.349559,-0.597625,-0.601580
finishing,0.329298,0.286684,0.576860,1.000000,0.373536,0.579365,0.851339,0.784659,0.691201,0.632248,0.337601,0.719709,0.528942,0.509506,0.554235,0.354473,0.394012,0.726982,0.008953,0.346688,-0.053048,0.806005,0.043714,-0.149366,0.798441,0.651455,0.720388,-0.284603,-0.229699,-0.260890,-0.479162,-0.465099,-0.287681,-0.470673,-0.473173
heading_accuracy,0.314099,0.207706,0.369747,0.373536,1.000000,0.548700,0.391638,0.400494,0.321120,0.307587,0.358634,0.551049,0.200077,0.267273,0.069220,0.295348,0.078953,0.540868,0.286803,0.477466,0.493051,0.406758,0.575705,0.452849,0.411554,0.337184,0.431336,0.461543,0.480624,0.441769,-0.665365,-0.647876,-0.391540,-0.647738,-0.651373
short_passing,0.458361,0.382605,0.790150,0.579365,0.548700,1.000000,0.640253,0.787994,0.732295,0.692461,0.797488,0.890042,0.503266,0.491380,0.510591,0.459289,0.461714,0.721767,0.060146,0.609956,0.092084,0.728791,0.453783,0.424288,0.677577,0.765336,0.609537,0.350171,0.415678,0.380990,-0.693313,-0.689528,-0.413317,-0.690702,-0.692926
volleys,0.361470,0.301302,0.637891,0.851339,0.391638,0.640253,1.000000,0.784366,0.752248,0.682751,0.414123,0.749557,0.513250,0.494090,0.559756,0.397233,0.415812,0.746484,0.023230,0.382649,-0.036108,0.814433,0.127922,-0.037604,0.778337,0.689852,0.711830,-0.168972,-0.106926,-0.126398,-0.508823,-0.487015,-0.279030,-0.490968,-0.492979
dribbling,0.354324,0.338675,0.809061,0.784659,0.400494,0.787994,0.784366,1.000000,0.810404,0.705863,0.574307,0.900768,0.698161,0.669503,0.703083,0.377661,0.546064,0.744135,0.008559,0.525359,-0.111598,0.806329,0.202770,0.106738,0.793309,0.732742,0.657698,0.004128,0.066958,0.046042,-0.652739,-0.650525,-0.427115,-0.653410,-0.656030
curve,0.357772,0.296073,0.789132,0.691201,0.321120,0.732295,0.752248,0.810404,1.000000,0.797411,0.585693,0.798737,0.549363,0.516899,0.619253,0.392910,0.493112,0.694749,-0.016958,0.454334,-0.114361,0.783548,0.203647,0.136265,0.720219,0.726943,0.648502,0.033682,0.095210,0.081215,-0.557165,-0.545760,-0.333487,-0.550659,-0.552295
free_kick_accuracy,0.349592,0.263358,0.707576,0.632248,0.307587,0.692461,0.682751,0.705863,0.797411,1.000000,0.599103,0.719743,0.430791,0.394848,0.504998,0.368100,0.430410,0.682790,-0.033263,0.416123,-0.056184,0.772652,0.232429,0.178005,0.654151,0.697390,0.665831,0.074506,0.134508,0.106969,-0.498800,-0.492137,-0.271076,-0.494807,-0.496332


In [43]:
# replace all values with their absolute
attributes_corr = attributes_corr.apply(np.absolute)

In [44]:
n = len(attributes_corr)

x1, x2, x3, x4, x5 = 0, 0, 0, 0, 0
x1_attr, x2_attr, x3_attr, x4_attr, x5_attr = 0, 0, 0, 0, 0

In [45]:
for i in range(n - 1):
    for j in range(i+1, n):
        if attributes_corr.iloc[j, i] >= x1:
            x5 = x4
            x5_attr = x4_attr
            x4 = x3
            x4_attr = x3_attr
            x3 = x2
            x3_attr = x2_attr
            x2 = x1
            x2_attr = x1_attr
            x1 = attributes_corr.iloc[j, i]
            x1_attr = (attributes_corr.index[j], attributes_corr.columns[i])
        elif attributes_corr.iloc[j, i] >= x2:
            x5 = x4
            x5_attr = x4_attr
            x4 = x3
            x4_attr = x3_attr
            x3 = x2
            x3_attr = x2_attr
            x2 = attributes_corr.iloc[j, i]
            x2_attr = (attributes_corr.index[j], attributes_corr.columns[i])
        elif attributes_corr.iloc[j, i] >= x3:
            x5 = x4
            x5_attr = x4_attr
            x4 = x3
            x4_attr = x3_attr
            x3 = attributes_corr.iloc[j, i]
            x3_attr = (attributes_corr.index[j], attributes_corr.columns[i])
        elif attributes_corr.iloc[j, i] >= x4:
            x5 = x4
            x5_attr = x4_attr
            x4 = attributes_corr.iloc[j, i]
            x4_attr = (attributes_corr.index[j], attributes_corr.columns[i])
        elif attributes_corr.iloc[j, i] >= x5:
            x5 = attributes_corr.iloc[j, i]
            x5_attr = (attributes_corr.index[j], attributes_corr.columns[i])          
            

In [46]:
print(x1_attr, x2_attr, x3_attr, x4_attr, x5_attr)

('gk_reflexes', 'gk_positioning') ('gk_reflexes', 'gk_handling') ('gk_positioning', 'gk_handling') ('sliding_tackle', 'standing_tackle') ('standing_tackle', 'marking')


In [47]:
x1, x2, x3, x4, x5

(0.9667111235984718,
 0.9663255745397531,
 0.9663005139500525,
 0.9531036770427397,
 0.9501336912446234)

## Task 10

**Find top-5 most similar players to Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [48]:
attributes.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [49]:
# drop duplicated players and leave onle recent values
attributes_10 = attributes.sort_values('date', ascending=False).drop_duplicates(subset='player_api_id', keep='first')
attributes_10.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
95620,95621,208127,307224,2016-07-07 00:00:00,64.0,68.0,right,medium,low,44.0,63.0,73.0,49.0,52.0,52.0,42.0,31.0,45.0,63.0,71.0,70.0,59.0,57.0,48.0,69.0,48.0,73.0,86.0,53.0,55.0,22.0,66.0,55.0,65.0,22.0,22.0,25.0,12.0,12.0,7.0,11.0,12.0
57229,57230,197948,184521,2016-07-07 00:00:00,73.0,77.0,right,medium,high,51.0,42.0,75.0,68.0,42.0,47.0,47.0,41.0,67.0,62.0,49.0,55.0,52.0,67.0,47.0,77.0,55.0,74.0,92.0,66.0,77.0,72.0,46.0,32.0,43.0,74.0,75.0,69.0,11.0,15.0,15.0,12.0,7.0
181048,181049,221274,512726,2016-07-07 00:00:00,63.0,72.0,right,medium,medium,51.0,66.0,55.0,57.0,60.0,64.0,50.0,39.0,48.0,63.0,80.0,80.0,72.0,60.0,70.0,64.0,74.0,67.0,61.0,61.0,33.0,22.0,56.0,48.0,59.0,15.0,16.0,12.0,11.0,12.0,12.0,12.0,7.0
178639,178640,216437,450002,2016-07-07 00:00:00,74.0,82.0,right,medium,medium,34.0,21.0,73.0,57.0,36.0,45.0,33.0,29.0,50.0,52.0,56.0,61.0,50.0,70.0,48.0,40.0,72.0,62.0,84.0,26.0,79.0,71.0,25.0,31.0,45.0,76.0,78.0,74.0,15.0,11.0,11.0,10.0,11.0
153508,153509,172555,45400,2016-07-07 00:00:00,68.0,70.0,right,medium,low,33.0,70.0,56.0,63.0,60.0,63.0,55.0,27.0,56.0,66.0,90.0,84.0,73.0,60.0,64.0,75.0,67.0,63.0,59.0,69.0,83.0,37.0,75.0,45.0,66.0,23.0,17.0,23.0,13.0,11.0,6.0,9.0,14.0


In [50]:
# leave only necassery columns
attributes_10 = attributes_10.iloc[:, 2:].drop(['date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], axis=1)
attributes_10.head()

,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
95620,307224,64.0,68.0,44.0,63.0,73.0,49.0,52.0,52.0,42.0,31.0,45.0,63.0,71.0,70.0,59.0,57.0,48.0,69.0,48.0,73.0,86.0,53.0,55.0,22.0,66.0,55.0,65.0,22.0,22.0,25.0,12.0,12.0,7.0,11.0,12.0
57229,184521,73.0,77.0,51.0,42.0,75.0,68.0,42.0,47.0,47.0,41.0,67.0,62.0,49.0,55.0,52.0,67.0,47.0,77.0,55.0,74.0,92.0,66.0,77.0,72.0,46.0,32.0,43.0,74.0,75.0,69.0,11.0,15.0,15.0,12.0,7.0
181048,512726,63.0,72.0,51.0,66.0,55.0,57.0,60.0,64.0,50.0,39.0,48.0,63.0,80.0,80.0,72.0,60.0,70.0,64.0,74.0,67.0,61.0,61.0,33.0,22.0,56.0,48.0,59.0,15.0,16.0,12.0,11.0,12.0,12.0,12.0,7.0
178639,450002,74.0,82.0,34.0,21.0,73.0,57.0,36.0,45.0,33.0,29.0,50.0,52.0,56.0,61.0,50.0,70.0,48.0,40.0,72.0,62.0,84.0,26.0,79.0,71.0,25.0,31.0,45.0,76.0,78.0,74.0,15.0,11.0,11.0,10.0,11.0
153508,45400,68.0,70.0,33.0,70.0,56.0,63.0,60.0,63.0,55.0,27.0,56.0,66.0,90.0,84.0,73.0,60.0,64.0,75.0,67.0,63.0,59.0,69.0,83.0,37.0,75.0,45.0,66.0,23.0,17.0,23.0,13.0,11.0,6.0,9.0,14.0


In [51]:
# find Neymar in Players
players[players.player_name.str.contains(pat = 'Neymar')]

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
7855,7867,19533,Neymar,190871,1992-02-05,175.26,150


In [52]:
# find Neymar's api_id
neymar_id = players.loc[players.player_name.str.contains(pat = 'Neymar'), 'player_api_id'].item()
neymar_id

19533

In [53]:
# variable 'neymar' contains Neymar's attributes
neymar = attributes_10[attributes_10.player_api_id == neymar_id]
neymar

,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
131464,19533,90.0,94.0,72.0,88.0,62.0,78.0,83.0,94.0,78.0,79.0,74.0,93.0,91.0,90.0,92.0,86.0,84.0,78.0,61.0,79.0,45.0,73.0,56.0,36.0,89.0,79.0,81.0,21.0,24.0,33.0,9.0,9.0,15.0,15.0,11.0


In [54]:
# create DF with players' attributes (excluding Neymar)
attr_without_neymar = attributes_10[attributes_10.player_api_id != neymar_id]
attr_without_neymar.head()

,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
95620,307224,64.0,68.0,44.0,63.0,73.0,49.0,52.0,52.0,42.0,31.0,45.0,63.0,71.0,70.0,59.0,57.0,48.0,69.0,48.0,73.0,86.0,53.0,55.0,22.0,66.0,55.0,65.0,22.0,22.0,25.0,12.0,12.0,7.0,11.0,12.0
57229,184521,73.0,77.0,51.0,42.0,75.0,68.0,42.0,47.0,47.0,41.0,67.0,62.0,49.0,55.0,52.0,67.0,47.0,77.0,55.0,74.0,92.0,66.0,77.0,72.0,46.0,32.0,43.0,74.0,75.0,69.0,11.0,15.0,15.0,12.0,7.0
181048,512726,63.0,72.0,51.0,66.0,55.0,57.0,60.0,64.0,50.0,39.0,48.0,63.0,80.0,80.0,72.0,60.0,70.0,64.0,74.0,67.0,61.0,61.0,33.0,22.0,56.0,48.0,59.0,15.0,16.0,12.0,11.0,12.0,12.0,12.0,7.0
178639,450002,74.0,82.0,34.0,21.0,73.0,57.0,36.0,45.0,33.0,29.0,50.0,52.0,56.0,61.0,50.0,70.0,48.0,40.0,72.0,62.0,84.0,26.0,79.0,71.0,25.0,31.0,45.0,76.0,78.0,74.0,15.0,11.0,11.0,10.0,11.0
153508,45400,68.0,70.0,33.0,70.0,56.0,63.0,60.0,63.0,55.0,27.0,56.0,66.0,90.0,84.0,73.0,60.0,64.0,75.0,67.0,63.0,59.0,69.0,83.0,37.0,75.0,45.0,66.0,23.0,17.0,23.0,13.0,11.0,6.0,9.0,14.0


In [55]:
attr_without_neymar = attr_without_neymar.dropna()

In [56]:
attr_without_neymar.shape

(10579, 36)

In [57]:
neymar.iloc[:, 10].item()

79.0

In [58]:
sq_ed = 0

In [59]:
def ed(row):
    sq_ed = 0
    for i in range(1, 36):
        sq_ed += np.square(neymar.iloc[:, i].item() - row.iloc[i].item())
    return np.sqrt(sq_ed)
    

In [60]:
attr_without_neymar['ed'] = attr_without_neymar.apply(ed, axis='columns')
attr_without_neymar.sort_values('ed').head()

,player_api_id,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,ed
140773,325916,81.0,87.0,72.0,87.0,60.0,78.0,80.0,89.0,81.0,82.0,58.0,90.0,90.0,85.0,91.0,82.0,85.0,77.0,69.0,79.0,55.0,83.0,48.0,31.0,85.0,79.0,76.0,15.0,21.0,21.0,9.0,11.0,9.0,10.0,11.0,33.837849
47247,107417,88.0,90.0,80.0,81.0,57.0,85.0,79.0,94.0,82.0,79.0,82.0,90.0,93.0,87.0,92.0,86.0,90.0,79.0,59.0,74.0,63.0,82.0,54.0,41.0,84.0,86.0,86.0,25.0,27.0,22.0,11.0,12.0,6.0,8.0,8.0,34.957117
64607,41157,79.0,80.0,74.0,81.0,61.0,73.0,79.0,85.0,73.0,80.0,71.0,85.0,87.0,86.0,89.0,80.0,89.0,80.0,65.0,68.0,45.0,79.0,53.0,28.0,80.0,70.0,70.0,21.0,28.0,23.0,6.0,9.0,11.0,7.0,5.0,38.314488
16460,30834,89.0,89.0,80.0,85.0,51.0,85.0,86.0,93.0,86.0,83.0,73.0,90.0,90.0,90.0,91.0,91.0,91.0,86.0,61.0,73.0,65.0,90.0,47.0,39.0,88.0,84.0,80.0,29.0,26.0,26.0,10.0,8.0,11.0,5.0,15.0,40.024992
102482,30981,94.0,94.0,80.0,93.0,71.0,88.0,85.0,96.0,89.0,90.0,79.0,96.0,95.0,90.0,92.0,92.0,95.0,80.0,68.0,75.0,59.0,88.0,48.0,22.0,90.0,90.0,74.0,13.0,23.0,21.0,6.0,11.0,15.0,14.0,8.0,43.543082


In [61]:
neymar_similarities = attr_without_neymar.sort_values('ed').head().player_api_id.to_list()

In [62]:
neymar_similarities

[325916, 107417, 41157, 30834, 30981]

In [63]:
players.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08,182.88,154


In [64]:
players[players.player_api_id.isin(neymar_similarities)].player_name

948           Arjen Robben
2835           Eden Hazard
3869    Giovani dos Santos
6169          Lionel Messi
8423          Paulo Dybala
Name: player_name, dtype: object

## Task 11

**Calculate the number of home matches played by the Borussia Dortmund team in Germany 1. Bundesliga in season 2008/2009**

In [65]:
leagues.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


In [66]:
league_id = leagues[leagues.name == 'Germany 1. Bundesliga'].id.item()
league_id

7809

In [67]:
# download Team dataset
teams = pd.read_sql("SELECT * FROM Team;", db)
teams.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [68]:
team_api_id = teams[teams.team_long_name == 'Borussia Dortmund'].team_api_id.item()
team_api_id

9789

In [69]:
matches.head(3)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.4,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.8,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.0,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.2,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.9,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.8,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.3,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.5,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.5,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75


In [70]:
borussia_match = matches[(matches.league_id == league_id) & (matches.season == '2008/2009') & (matches.home_team_api_id == team_api_id)]
borussia_match.head(3)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
7828,7829,7809,7809,2008/2009,11,2008-11-02 00:00:00,499409,9789,9911,1,1,1.0,2.0,4.0,6.0,8.0,5.0,4.0,6.0,5.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,5.0,7.0,7.0,9.0,11.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,27358.0,414794.0,71399.0,36388.0,30255.0,30707.0,40190.0,27363.0,35990.0,43319.0,34102.0,31299.0,39898.0,39892.0,17312.0,43061.0,36008.0,NaN,30256.0,33336.0,30652.0,96209.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><corners>1</corners></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>68</comment><event...,1.55,3.80,6.25,1.50,3.8,5.85,1.50,3.8,5.5,1.50,3.5,6.00,NaN,NaN,NaN,1.53,3.6,5.0,2.00,3.40,3.75,1.55,3.4,5.5,1.53,3.75,6.0,1.57,3.5,5.50
7845,7846,7809,7809,2008/2009,13,2008-11-15 00:00:00,499426,9789,9810,4,0,1.0,2.0,4.0,6.0,8.0,5.0,4.0,6.0,5.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,2.0,8.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,5.0,7.0,7.0,9.0,11.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,10.0,10.0,27358.0,30255.0,71399.0,20396.0,414794.0,30707.0,40190.0,27363.0,36132.0,43319.0,31718.0,27467.0,NaN,27475.0,30572.0,27484.0,36060.0,38863.0,41923.0,41034.0,37760.0,69864.0,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card />,<cross />,<corner />,<possession />,1.65,3.75,5.50,1.55,3.5,5.90,1.65,3.5,4.5,1.67,3.4,4.33,NaN,NaN,NaN,1.60,3.4,4.8,1.67,3.50,5.50,1.60,3.8,5.0,1.70,3.60,5.0,1.67,3.5,4.75
7863,7864,7809,7809,2008/2009,15,2008-11-30 00:00:00,499444,9789,8721,0,0,1.0,2.0,4.0,6.0,8.0,5.0,4.0,6.0,5.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,5.0,7.0,7.0,9.0,11.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,27358.0,414794.0,71399.0,36388.0,94685.0,40190.0,27363.0,27365.0,36132.0,34102.0,35990.0,30820.0,30906.0,30914.0,30902.0,37317.0,31290.0,19511.0,27339.0,39902.0,27324.0,15403.0,<goal />,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />,2.20,3.30,3.25,2.15,3.2,3.10,2.20,3.2,2.9,2.10,3.2,3.00,NaN,NaN,NaN,2.10,3.2,3.0,2.10,3.25,3.25,2.20,3.4,3.0,2.20,3.25,3.1,2.20,3.3,3.00


In [71]:
borussia_bundesliga_2008_2009_matches = borussia_match.match_api_id.nunique()
borussia_bundesliga_2008_2009_matches

17

## Task 12

**Find a team having the most matches (both home and away!) in the Germany 1. Bundesliga in 2008/2009 season. Return number of matches.**

In [72]:
bundesliga_08_09 = matches[(matches.league_id == league_id) & (matches.season == '2008/2009')]
bundesliga_08_09.head(3)

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
7808,7809,7809,7809,2008/2009,1,2008-08-15 00:00:00,499317,9823,9790,2,2,1.0,2.0,4.0,6.0,8.0,5.0,4.0,6.0,5.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,3.0,5.0,7.0,3.0,5.0,7.0,1.0,3.0,3.0,3.0,3.0,5.0,7.0,7.0,9.0,11.0,11.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,10.0,10.0,10.0,27284.0,35988.0,39774.0,33085.0,30894.0,38244.0,30872.0,38843.0,95078.0,30638.0,32118.0,25524.0,36183.0,27293.0,37787.0,37156.0,30749.0,30598.0,39106.0,38216.0,33101.0,30764.0,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />,1.57,3.8,6.0,1.55,3.70,5.45,1.6,3.6,4.8,1.53,3.50,5.5,NaN,NaN,NaN,1.53,3.5,5.5,1.62,3.40,5.50,1.55,3.50,5.50,1.60,3.6,5.75,1.53,3.75,5.5
7809,7810,7809,7809,2008/2009,1,2008-08-16 00:00:00,499318,8178,9789,2,3,1.0,2.0,4.0,6.0,8.0,5.0,3.0,5.0,7.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,5.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,36147.0,27427.0,27492.0,128037.0,28081.0,27429.0,34681.0,49939.0,30937.0,27556.0,31199.0,27358.0,79737.0,71399.0,36388.0,27360.0,43319.0,36132.0,30707.0,27363.0,31718.0,27514.0,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />,2.05,3.3,3.6,2.05,3.25,3.30,2.0,3.3,3.2,1.91,3.25,3.4,NaN,NaN,NaN,1.95,3.1,3.5,1.91,3.25,3.75,2.00,3.25,3.25,2.05,3.3,3.50,2.00,3.25,3.4
7810,7811,7809,7809,2008/2009,1,2008-08-16 00:00:00,499319,10189,9904,3,0,1.0,2.0,4.0,6.0,8.0,3.0,5.0,7.0,4.0,6.0,5.0,1.0,2.0,4.0,6.0,8.0,3.0,5.0,7.0,5.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,6.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,5.0,5.0,5.0,8.0,10.0,10.0,37322.0,27461.0,68253.0,27301.0,30250.0,30251.0,27483.0,27307.0,35997.0,25773.0,30249.0,27341.0,30811.0,27286.0,NaN,25483.0,27351.0,27349.0,39045.0,34400.0,30854.0,39840.0,<goal><value><comment>n</comment><stats><goals...,<shoton />,<shotoff />,<foulcommit />,<card><value><comment>y</comment><stats><ycard...,<cross />,<corner />,<possession />,1.57,3.8,6.0,1.60,3.65,5.00,1.6,3.6,4.8,1.57,3.50,5.0,NaN,NaN,NaN,1.60,3.4,4.8,1.62,3.40,5.50,1.50,3.60,6.00,1.60,3.6,5.60,1.57,3.75,5.0


In [73]:
bundesliga_08_09.shape

(306, 115)

In [74]:
bundesliga_08_09.match_api_id.nunique()

306

In [75]:
home_matches = bundesliga_08_09.home_team_api_id.value_counts()

In [76]:
away_matches = bundesliga_08_09.away_team_api_id.value_counts()

In [77]:
# create DF with number of matches
number_of_matches = pd.merge(home_matches, away_matches, left_index=True, right_index=True)

In [78]:
number_of_matches.head()

,home_team_api_id,away_team_api_id
8697,17,17
8178,17,17
8722,17,17
10269,17,17
8226,17,17


In [79]:
number_of_matches = number_of_matches.rename(columns={'home_team_api_id': 'home_matches_count',
                                 'away_team_api_id': 'away_matches_count'})
number_of_matches.head()

,home_matches_count,away_matches_count
8697,17,17
8178,17,17
8722,17,17
10269,17,17
8226,17,17


In [80]:
number_of_matches['total_matches_count'] = number_of_matches.home_matches_count + number_of_matches.away_matches_count
number_of_matches.head()

,home_matches_count,away_matches_count,total_matches_count
8697,17,17,34
8178,17,17,34
8722,17,17,34
10269,17,17,34
8226,17,17,34


In [81]:
number_of_matches.total_matches_count.max()

34

In [82]:
team_most_matches_bundesliga_2008_2009 = number_of_matches.total_matches_count.max()

## Task 13

**Count total number of Arsenal matches (both home and away!) in the 2015/2016 season which they have won. <br><br>
Note:** Winning a game means scoring **more** goals than an opponent.

In [83]:
teams.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [84]:
arsenal_id = teams[teams.team_long_name == 'Arsenal'].team_api_id.item()
arsenal_id

9825

In [85]:
arsenal_15_16_home = matches[(matches.season == '2015/2016') & (matches.home_team_api_id == arsenal_id)]

In [86]:
arsenal_15_16_home.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
4388,4389,1729,1729,2015/2016,1,2015-08-09 00:00:00,1987032,9825,8654,0,2,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,3.0,5.0,7.0,5.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,6.0,8.0,10.0,10.0,30859.0,26154.0,35606.0,46539.0,38521.0,159594.0,37436.0,75489.0,36378.0,196386.0,46469.0,169756.0,109897.0,35110.0,49543.0,155782.0,37169.0,575789.0,148302.0,25496.0,18506.0,192899.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>56</comment><stats...,1.29,6.00,12.0,1.28,5.75,10.50,1.33,4.8,8.30,1.29,5.5,12.0,1.31,5.75,12.00,1.30,5.0,11.0,NaN,NaN,NaN,1.30,5.75,12.0,NaN,NaN,NaN,NaN,NaN,NaN
4398,4399,1729,1729,2015/2016,10,2015-10-24 00:00:00,1988800,9825,8668,2,1,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,30859.0,427438.0,177503.0,46539.0,38521.0,159594.0,37436.0,196386.0,36378.0,50047.0,46469.0,31465.0,77690.0,263653.0,23268.0,316688.0,101192.0,23782.0,251925.0,215384.0,30895.0,181276.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>60</comment><stats...,1.44,4.75,8.0,1.44,4.60,7.25,1.45,4.0,7.00,1.44,4.5,8.0,1.43,5.07,8.20,1.44,4.0,8.0,NaN,NaN,NaN,1.44,4.80,8.0,NaN,NaN,NaN,NaN,NaN,NaN
4418,4419,1729,1729,2015/2016,12,2015-11-08 00:00:00,1988820,9825,8586,1,1,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,30859.0,26154.0,35606.0,46539.0,38521.0,159594.0,37436.0,242094.0,36378.0,50047.0,46469.0,26295.0,159833.0,160599.0,37762.0,46353.0,361315.0,363333.0,174850.0,41008.0,157723.0,194165.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><cor

In [87]:
home_wins = arsenal_15_16_home[arsenal_15_16_home.home_team_goal > arsenal_15_16_home.away_team_goal].match_api_id.nunique()

In [88]:
arsenal_15_16_away = matches[(matches.season == '2015/2016') & (matches.away_team_api_id == arsenal_id)] 
arsenal_15_16_away.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
4414,4415,1729,1729,2015/2016,11,2015-10-31 00:00:00,1988816,10003,9825,0,3,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,30973.0,111800.0,155050.0,24948.0,102356.0,144996.0,127130.0,95955.0,157729.0,52563.0,26344.0,30859.0,427438.0,35606.0,46539.0,38521.0,159594.0,37436.0,242094.0,36378.0,50047.0,46469.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><blocked>1</blocked></st...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>58</comment><stats...,5.0,4.00,1.75,4.50,3.60,1.75,4.7,3.6,1.70,4.75,3.80,1.73,5.05,3.93,1.76,5.0,3.50,1.75,NaN,NaN,NaN,5.00,4.00,1.73,NaN,NaN,NaN,NaN,NaN,NaN
4437,4438,1729,1729,2015/2016,13,2015-11-21 00:00:00,1988839,8659,9825,2,1,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,23021.0,169162.0,26527.0,43248.0,23257.0,113968.0,24148.0,24171.0,33468.0,175947.0,51545.0,30859.0,427438.0,35606.0,46539.0,38521.0,37436.0,159594.0,50047.0,36378.0,78513.0,46469.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><event_incident_typefk>139</eve...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>32</comment><stats...,7.0,4.33,1.53,6.00,4.33,1.53,5.6,3.9,1.55,6.50,4.33,1.53,6.68,4.29,1.57,6.5,4.00,1.53,NaN,NaN,NaN,7.00,4.33,1.53,NaN,NaN,NaN,NaN,NaN,NaN
4444,4445,1729,1729,2015/2016,14,2015-11-29 00:00:00,1988846,9850,9825,1,1,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,8.0,8.0,8.0,11.0,40602.0,265918.0,72417.0,26209.0,25667.0,24393.0,32734.0,46354.0,35493.0,231801.0,37779.0,30859.0,427438.0,35606.0,46539.0,38521.0,23688.0,37436.0,75489.0,36378.0,50047.0,46469.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</c

In [89]:
away_wins = arsenal_15_16_away[arsenal_15_16_away.away_team_goal > arsenal_15_16_away.home_team_goal].match_api_id.nunique()

In [90]:
arsenal_won_matches_2015_2016 = home_wins + away_wins
arsenal_won_matches_2015_2016

20

## Task 14

**Find a team with the highest win rate in the 2015/2016 season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order**

In [91]:
# leave only matches of season 2015/2016
matches_15_16 = matches[matches.season == '2015/2016'][['league_id', 'season', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']]

In [92]:
matches_15_16.head()

,league_id,season,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
1488,1,2015/2016,1979832,9997,8342,2,1
1489,1,2015/2016,1979833,8571,9985,2,1
1490,1,2015/2016,1979834,9987,1773,3,1
1491,1,2015/2016,1979835,8573,8203,3,1
1492,1,2015/2016,1979836,10000,9994,3,1


In [93]:
matches_15_16['home_win'] = matches_15_16.apply(lambda x: 1 if x['home_team_goal'] > x['away_team_goal'] else 0, axis=1)

In [94]:
matches_15_16['away_win'] = matches_15_16.apply(lambda x: 1 if x['away_team_goal'] > x['home_team_goal'] else 0, axis=1)

In [95]:
matches_15_16.head()

,league_id,season,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_win,away_win
1488,1,2015/2016,1979832,9997,8342,2,1,1,0
1489,1,2015/2016,1979833,8571,9985,2,1,1,0
1490,1,2015/2016,1979834,9987,1773,3,1,1,0
1491,1,2015/2016,1979835,8573,8203,3,1,1,0
1492,1,2015/2016,1979836,10000,9994,3,1,1,0


In [96]:
# count total number of home mathes and wins by each team
home_matches = (matches_15_16.groupby('home_team_api_id').agg({'match_api_id': 'nunique', 'home_win': 'sum'})
                             .rename(columns={'match_api_id': 'home_matches_count',
                                             'home_win': 'home_win_count'}))
home_matches.head()

,home_matches_count,home_win_count
home_team_api_id,,
1601,15,6
1773,15,6
1957,15,6
2182,15,6
2186,15,7


In [97]:
# count total number of away mathes and wins by each team
away_matches = (matches_15_16.groupby('away_team_api_id').agg({'match_api_id': 'nunique', 'away_win': 'sum'})
                             .rename(columns={'match_api_id': 'away_matches_count',
                                             'away_win': 'away_win_count'}))
away_matches.head()

,away_matches_count,away_win_count
away_team_api_id,,
1601,15,5
1773,15,1
1957,15,4
2182,15,7
2186,15,5


In [98]:
total_matches = pd.merge(home_matches, away_matches, left_index=True, right_index=True)
total_matches.head()

,home_matches_count,home_win_count,away_matches_count,away_win_count
home_team_api_id,,,,
1601,15,6,15,5
1773,15,6,15,1
1957,15,6,15,4
2182,15,6,15,7
2186,15,7,15,5


In [99]:
total_matches['total_matches_count'] = total_matches.home_matches_count + total_matches.away_matches_count

In [100]:
total_matches['total_wins_count'] = total_matches.home_win_count + total_matches.away_win_count
total_matches.head()

,home_matches_count,home_win_count,away_matches_count,away_win_count,total_matches_count,total_wins_count
home_team_api_id,,,,,,
1601,15,6,15,5,30,11
1773,15,6,15,1,30,7
1957,15,6,15,4,30,10
2182,15,6,15,7,30,13
2186,15,7,15,5,30,12


In [101]:
total_matches['win_rate'] = total_matches.total_wins_count / total_matches.total_matches_count
total_matches.sort_values('win_rate', ascending=False).head()

,home_matches_count,home_win_count,away_matches_count,away_win_count,total_matches_count,total_wins_count,win_rate
home_team_api_id,,,,,,,
9772,17,15,17,14,34,29,0.852941
9823,17,15,17,13,34,28,0.823529
9768,17,13,17,14,34,27,0.794118
9847,19,15,19,15,38,30,0.789474
8640,17,13,17,13,34,26,0.764706


In [102]:
win_team_id = total_matches.win_rate.idxmax()

In [103]:
teams.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [104]:
team_highest_winrate_2015_2016 = teams[teams.team_api_id == win_team_id].team_long_name.item()
team_highest_winrate_2015_2016

'SL Benfica'

## Task 15

**Determine the team with the maximum days' gap between matches in England Premier League 2010/2011 season. Return number of days in that gap. <br>
Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [105]:
england_premier_id = leagues[leagues.name == 'England Premier League'].id.item()

In [106]:
england_premier_10_11 = matches[(matches.season == '2010/2011') & (matches.league_id == england_premier_id)]

In [107]:
england_premier_10_11.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
2488,2489,1729,1729,2010/2011,1,2010-08-14 00:00:00,839796,10252,8654,3,0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,5.0,1.0,2.0,4.0,6.0,8.0,1.0,3.0,5.0,7.0,9.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,9.0,11.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,7.0,11.0,30380.0,30357.0,161414.0,24211.0,24136.0,139671.0,30892.0,38609.0,38807.0,23354.0,26165.0,36374.0,35110.0,109897.0,23818.0,26348.0,36394.0,24223.0,37169.0,34590.0,30734.0,34543.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><corners>1</corners></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>67</comment><event...,2.00,3.30,4.00,1.85,3.40,4.10,1.80,3.4,4.0,1.83,3.4,4.33,NaN,NaN,NaN,1.91,3.25,4.33,2.00,3.40,3.75,2.00,3.25,4.20,2.00,3.25,3.75,1.80,3.40,4.33
2489,2490,1729,1729,2010/2011,1,2010-08-14 00:00:00,839797,8655,8668,1,0,1.0,2.0,4.0,6.0,8.0,1.0,3.0,5.0,7.0,9.0,5.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,7.0,11.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,9.0,11.0,30622.0,30658.0,19020.0,23921.0,30739.0,23927.0,186137.0,30342.0,23916.0,25667.0,21613.0,31465.0,30371.0,23268.0,24216.0,24846.0,24006.0,39618.0,24011.0,36012.0,30876.0,30735.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>45</comment><event...,2.88,3.25,2.50,2.75,3.25,2.45,2.55,3.2,2.5,2.80,3.3,2.50,NaN,NaN,NaN,2.88,3.20,2.50,2.75,3.30,2.50,3.10,3.30,2.38,2.90,3.20,2.40,2.70,3.25,2.50
2490,2491,1729,1729,2010/2011,1,2010-08-14 00:00:00,839799,8559,9879,0,0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,8.0,6.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,9.0,11.0,23932.0,26454.0,23783.0,40128.0,24728.0,130670.0,35532.0,33633.0,24455.0,34261.0,23934.0,35477.0,37266.0,23780.0,26777.0,33045.0,24020.0,25253.0,30338.0,34574.0,24737.0,24741.0,<goal />,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>5

In [108]:
home_teams = england_premier_10_11[['match_api_id', 'date', 'home_team_api_id']].rename(columns={'home_team_api_id': 'team_api_id'})
home_teams.head()

,match_api_id,date,team_api_id
2488,839796,2010-08-14 00:00:00,10252
2489,839797,2010-08-14 00:00:00,8655
2490,839799,2010-08-14 00:00:00,8559
2491,839800,2010-08-14 00:00:00,8455
2492,839801,2010-08-15 00:00:00,8650


In [109]:
home_teams.shape

(380, 3)

In [110]:
home_teams.match_api_id.nunique()

380

In [111]:
away_teams = england_premier_10_11[['match_api_id', 'date', 'away_team_api_id']].rename(columns={'away_team_api_id': 'team_api_id'})
away_teams.head()

,match_api_id,date,team_api_id
2488,839796,2010-08-14 00:00:00,8654
2489,839797,2010-08-14 00:00:00,8668
2490,839799,2010-08-14 00:00:00,9879
2491,839800,2010-08-14 00:00:00,8659
2492,839801,2010-08-15 00:00:00,9825


In [112]:
total_teams = pd.concat([home_teams, away_teams])
total_teams.head()

,match_api_id,date,team_api_id
2488,839796,2010-08-14 00:00:00,10252
2489,839797,2010-08-14 00:00:00,8655
2490,839799,2010-08-14 00:00:00,8559
2491,839800,2010-08-14 00:00:00,8455
2492,839801,2010-08-15 00:00:00,8650


In [123]:
df_total = total_teams[['team_api_id', 'date']].sort_values(['team_api_id', 'date'])
df_total.head()

,team_api_id,date
2491,8455,2010-08-14 00:00:00
2607,8455,2010-08-21 00:00:00
2717,8455,2010-08-28 00:00:00
2809,8455,2010-09-11 00:00:00
2827,8455,2010-09-19 00:00:00


In [124]:
df_total.dtypes

team_api_id     int64
date           object
dtype: object

In [125]:
df_total.date = pd.to_datetime(df_total.date)
df_total.dtypes

team_api_id             int64
date           datetime64[ns]
dtype: object

In [151]:
def date_differ(df):
    for i in range(1, len(df)):
        if df.iloc[i, 0] == df.iloc[(i - 1), 0]:
            df.iloc[i, 2] = df.iloc[i, 1] - df.iloc[(i - 1), 1]
        else: pass
    df.iloc[:, 2] = pd.to_timedelta(df.iloc[:, 2]).dt.days
    return df

In [152]:
df_total['date_diff'] = np.nan

In [153]:
df_total.head()

,team_api_id,date,date_diff
2491,8455,2010-08-14,NaN
2607,8455,2010-08-21,NaN
2717,8455,2010-08-28,NaN
2809,8455,2010-09-11,NaN
2827,8455,2010-09-19,NaN


In [154]:
total_adj = date_differ(df_total)
total_adj.head()

,team_api_id,date,date_diff
2491,8455,2010-08-14,NaN
2607,8455,2010-08-21,7.0
2717,8455,2010-08-28,7.0
2809,8455,2010-09-11,14.0
2827,8455,2010-09-19,8.0


In [156]:
max_gap = total_adj.date_diff.max()

In [157]:
total_adj[total_adj.date_diff == max_gap]

,team_api_id,date,date_diff
2608,8650,2010-12-29,18.0
2705,8658,2011-03-05,18.0


In [158]:
highest_gap_england_2010_2011 = total_adj.date_diff.max()